In [4]:
! pip install tensorflow


[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: C:\Users\notti\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Rescaling
from tensorflow.keras.layers import PReLU
from tensorflow.keras.applications import MobileNetV2

In [2]:
# Define dataset paths
train_data_dir = 'C:\\Users\\notti\\Downloads\\Gangz\\Dataset\\train'
test_data_dir = 'C:\\Users\\notti\\Downloads\\Gangz\\Dataset\\test'
valid_data_dir = 'C:\\Users\\notti\\Downloads\\Gangz\\Dataset\\valid'


In [6]:
# Define image dimensions and other hyperparameters
image_size = (224, 224)
batch_size = 16
num_classes = 389
num_epochs = 10
red_batch_size=16

In [4]:
# Function to preprocess and augment the images
def preprocess_image(image, label):
    # Normalize pixel values to [0,1]
    image = tf.image.convert_image_dtype(image, tf.float32)
    # Add your desired image augmentations here (e.g., flip, rotation, etc.)
    return image, label


In [7]:
# Preprocess and augment the training data
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    image_size=image_size,
    batch_size=red_batch_size,
    label_mode='categorical',
    validation_split=0.2,
    seed=123,
    subset='training'
)

train_data = train_data.map(preprocess_image).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

Found 62237 files belonging to 389 classes.
Using 49790 files for training.


In [8]:
# Preprocess the validation data
val_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_dir,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='categorical',
    validation_split=0.2,
    seed=123,
    subset='validation'
)

val_data = val_data.map(preprocess_image).cache().prefetch(buffer_size=tf.data.AUTOTUNE)


Found 62237 files belonging to 389 classes.
Using 12447 files for validation.


In [9]:
# Load a pre-trained model (e.g., MobileNet) without the top (fully connected) layers
base_model = tf.keras.applications.MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(image_size[0], image_size[1], 3)
    )
base_model.summary()

Model: "mobilenetv2_1.00_224"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 Conv1 (Conv2D)              (None, 112, 112, 32)         864       ['input_1[0][0]']             
                                                                                                  
 bn_Conv1 (BatchNormalizati  (None, 112, 112, 32)         128       ['Conv1[0][0]']               
 on)                                                                                              
                                                                                                  
 Conv1_relu (ReLU)           (None, 112, 112, 32)         0         ['bn_Conv1[

In [10]:
# Freeze the base model's layers
base_model.trainable = False

In [11]:
# Add custom classification layers on top of the base model
model = keras.Sequential([
    Rescaling(1.0 / 255.0),
    base_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(128),
    PReLU(),
    keras.layers.Dense(num_classes, activation='softmax')
])

In [12]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import h5py

mod_chk = ModelCheckpoint(filepath = 'C:\\Users\\notti\\Downloads\\Gangz\\Dataset\\model.h5', monitor='val_loss', save_best_only=True)
lr_rate = ReduceLROnPlateau( monitor = 'val_loss', patience=6, factor=0.1)

In [14]:
# Train the model
model.fit(train_data, epochs=num_epochs, validation_data=val_data, callbacks=[mod_chk, lr_rate])

Epoch 1/10
3112/3112 [==============================] - ETA: 0s - loss: 1.7809 - accuracy: 0.5878

c:\Users\notti\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


3112/3112 [==============================] - 1305s 418ms/step - loss: 1.7809 - accuracy: 0.5878 - val_loss: 1.0132 - val_accuracy: 0.7329 - lr: 0.0010
Epoch 2/10
3112/3112 [==============================] - 1262s 406ms/step - loss: 0.7066 - accuracy: 0.8076 - val_loss: 0.8739 - val_accuracy: 0.7728 - lr: 0.0010
Epoch 3/10
3112/3112 [==============================] - 1272s 409ms/step - loss: 0.4866 - accuracy: 0.8669 - val_loss: 0.8874 - val_accuracy: 0.7854 - lr: 0.0010
Epoch 4/10
3112/3112 [==============================] - 1267s 407ms/step - loss: 0.3548 - accuracy: 0.9020 - val_loss: 0.9631 - val_accuracy: 0.7839 - lr: 0.0010
Epoch 5/10
3112/3112 [==============================] - 1262s 406ms/step - loss: 0.2688 - accuracy: 0.9230 - val_loss: 1.0251 - val_accuracy: 0.7838 - lr: 0.0010
Epoch 6/10
3112/3112 [==============================] - 1258s 404ms/step - loss: 0.2198 - accuracy: 0.9349 - val_loss: 1.1628 - val_accuracy: 0.7763 - lr: 0.0010
Epoch 7/10
3112/3112 [=================

In [7]:
import tensorflow as tf

model = tf.keras.models.load_model('model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("model.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\PADMAK~1\AppData\Local\Temp\tmpj35wiio_\assets


INFO:tensorflow:Assets written to: C:\Users\PADMAK~1\AppData\Local\Temp\tmpj35wiio_\assets


9724304